In [1]:
############################### DEPENDENCIES ##################################

import json
import os
import sys
import contextlib
import io
import numpy as np
import inspect
import os.path
import matplotlib.pyplot as plt
from math import pi, atan, sin, sqrt, tanh, cosh, exp, ceil
import seaborn as sns
from numpy.core.fromnumeric import shape
import random
import time
import math
import statistics
from pathlib import Path

In [11]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

print(module_path)

project_root = module_path

c:\Users\aurel\Documents\GitHub\Code_Thesis_GitHub\Code_Thesis_GitHub


In [6]:
#filename = inspect.getframeinfo(inspect.currentframe()).filename
#file_path = os.path.dirname(os.path.abspath(filename))
#project_root = os.path.dirname(file_path)

#if project_root not in sys.path:
#    sys.path.append(project_root)

#print(project_root)

C:\Users\aurel\AppData\Local\Temp


In [12]:
from event_model import event_model as em
from validator import validator_lite as vl
import data_analysis.event_generator as eg
from visual.color_map import Colormap

In [13]:
def load_event(file_name, plot_event=False):
# NB: some modifications might have been done for a better understanding
    f = open(file_name)
    json_data_event = json.loads(f.read())

    ev = em.event(json_data_event, read_tracks=True)

    modules = ev.modules
    tracks = ev.real_tracks

    if plot_event:
        eg.plot_tracks_and_modules(tracks, modules, title="Loaded Event")

    modules_even = []
    modules_odd = []

    for i in range(len(modules)):
        if i % 2 == 0:
            modules_even.append(modules[i])
        else:
            modules_odd.append(modules[i])

    return json_data_event, modules_even, modules_odd, tracks, modules

In [14]:
i = 2
event_file_name = "/datasets/minibias/velo_event_"
file_name = project_root + event_file_name
print(file_name)

c:\Users\aurel\Documents\GitHub\Code_Thesis_GitHub\Code_Thesis_GitHub/datasets/minibias/velo_event_


In [167]:
json_data_event, modules_even, modules_odd, tracks, modules = load_event(file_name + str(i) + ".json", plot_event=False)

In [168]:
counter_even = 0
counter_odd = 0
for module in modules_even:
    if module.hits() != []:
        counter_even +=1

print(f'Modules even empty: {counter_even}')
print(f'Modules even not empty: {len(modules_even)-counter_even}')

for module in modules_odd:
    if module.hits() != []:
        counter_odd +=1
print(f'Modules odd empty: {counter_odd}')
print(f'Module odd not empty: {len(modules_odd)-counter_odd}')


Modules even empty: 20
Modules even not empty: 6
Modules odd empty: 16
Module odd not empty: 10


In [169]:
print(tracks[0])

print(type(tracks))
print(type(tracks[0].hits))

#> look at each hits in a track, 
# if module number is even increase counter even of 1, 
# if module number is odd increase counter odd of 1


Track with 15 hits: [#15 module 23 {1.69, 6.806, 99.419}, #21 module 25 {1.962, 8.012, 124.419}, #27 module 27 {2.273, 9.178, 149.419}, #35 module 29 {2.565, 10.326, 174.419}, #43 module 31 {2.818, 11.512, 199.419}, #52 module 33 {3.129, 12.678, 224.419}, #63 module 35 {3.401, 13.884, 249.419}, #72 module 37 {3.712, 15.051, 274.419}, #79 module 39 {4.296, 17.423, 324.419}, #89 module 41 {5.151, 21.001, 399.419}, #99 module 43 {6.311, 25.753, 500.581}, #107 module 45 {7.439, 30.459, 600.581}, #115 module 47 {7.983, 32.792, 650.581}, #124 module 49 {8.567, 35.164, 700.581}, #132 module 51 {9.111, 37.498, 750.581}]
<class 'list'>
<class 'list'>


Pour chaque event, 
regarder chaque trace 
compter le nombre de hits en tout,
compter le nombre hits dans dans chaque type de module(even and odd)



In [200]:
# For 1 event
number_hits_by_track = []
 
counter_by_track_odd = []
counter_by_track_even =[]

number_of_tracks = len(tracks)
print(number_of_tracks)

for track in tracks:
    number_hits = len(track.hits)
    number_hits_by_track.append(number_hits)

for track in tracks:
    counter_hits_odd_module = 0
    counter_hits_even_module = 0
    for hits in track:
        if hits.module_number % 2 == 1:
            counter_hits_odd_module +=1
        else:
            counter_hits_even_module +=1
    counter_by_track_odd.append(counter_hits_odd_module)
    counter_by_track_even.append(counter_hits_even_module)

print(number_hits_by_track)
print(counter_by_track_odd)
print(counter_by_track_even)

percentage_of_tracks_odd_modules = [x/y for x,y in zip(counter_by_track_odd,number_hits_by_track)]
percentage_of_tracks_even_modules = [x/y for x,y in zip(counter_by_track_even,number_hits_by_track)]

print(f'Percentage in odd: {percentage_of_tracks_odd_modules}')
print(f'Percentage in even: {percentage_of_tracks_even_modules}')

number_of_not_mixed_tracks = len([x for x in percentage_of_tracks_even_modules if x in [0,1]])
percentage_of_not_mixed_tracks = number_of_not_mixed_tracks/number_of_tracks

print(number_of_not_mixed_tracks)
print(percentage_of_not_mixed_tracks)

number_of_mixed_tracks = len([x for x in percentage_of_tracks_even_modules if not x in [0,1]])
percentage_of_mixed_tracks = number_of_mixed_tracks/number_of_tracks

print(number_of_mixed_tracks)
print(percentage_of_mixed_tracks)





13
[15, 12, 8, 5, 10, 10, 3, 15, 11, 13, 11, 12, 5]
[15, 0, 0, 4, 0, 0, 1, 12, 0, 13, 11, 5, 0]
[0, 12, 8, 1, 10, 10, 2, 3, 11, 0, 0, 7, 5]
Percentage in odd: [1.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.3333333333333333, 0.8, 0.0, 1.0, 1.0, 0.4166666666666667, 0.0]
Percentage in even: [0.0, 1.0, 1.0, 0.2, 1.0, 1.0, 0.6666666666666666, 0.2, 1.0, 0.0, 0.0, 0.5833333333333334, 1.0]
9
0.6923076923076923
4
0.3076923076923077


In [201]:
# Create the fct for 1 event
def tracks_even_odd_modules(tracks):
    number_hits_by_track = []
    counter_by_track_odd = []
    counter_by_track_even =[]

    number_of_tracks = len(tracks)

    for track in tracks:
        number_hits = len(track.hits)
        number_hits_by_track.append(number_hits)

    for track in tracks:
        counter_hits_odd_module = 0
        counter_hits_even_module = 0
        for hits in track:
            if hits.module_number % 2 == 1:
                counter_hits_odd_module +=1
            else:
                counter_hits_even_module +=1
        counter_by_track_odd.append(counter_hits_odd_module)
        counter_by_track_even.append(counter_hits_even_module)

    percentage_of_tracks_odd_modules = [x/y for x,y in zip(counter_by_track_odd,number_hits_by_track)]
    percentage_of_tracks_even_modules = [x/y for x,y in zip(counter_by_track_even,number_hits_by_track)]

    number_of_not_mixed_tracks = len([x for x in percentage_of_tracks_even_modules if x in [0,1]])
    percentage_of_not_mixed_tracks = number_of_not_mixed_tracks/number_of_tracks

    number_of_mixed_tracks = len([x for x in percentage_of_tracks_even_modules if not x in [0,1]])
    percentage_of_mixed_tracks = number_of_mixed_tracks/number_of_tracks

    return  percentage_of_not_mixed_tracks, percentage_of_mixed_tracks


In [208]:
percentage_of_not_mixed_tracks, percentage_of_mixed_tracks = tracks_even_odd_modules(tracks)
print(f'The percentage of not mixed track in the event {i} is {percentage_of_not_mixed_tracks} %.')
print(f'The percentage of mixed track in the event {i} is {percentage_of_mixed_tracks} %.')

The percentage of not mixed track in the event 2 is 0.6923076923076923 %.
The percentage of mixed track in the event 2 is 0.3076923076923077 %.
